<a href="https://colab.research.google.com/github/LadyJ101/dsn_hackathon_project/blob/main/DSN_HACKATHON_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive') #Mounted my google drive to this notebook


MessageError: Error: credential propagation was unsuccessful

IMPORTING NECESSARY LIBRARIES

In [ ]:
#Installing XGBoost and optuna
print("Installing required packages with confirmation...")

# Install with clear output
!pip install xgboost optuna --quiet

# Verify installations
try:
    import xgboost as xgb
    import optuna
    print("XGBoost and Optuna installed successfully!")
    print(f"XGBoost version: {xgb.__version__}")
except Exception as e:
    print(f"Installation failed: {e}")
    print("Trying alternative installation...")
    !pip install xgboost optuna

In [ ]:

import pandas as pd #Pandas for data manipulation
import numpy as np #Numpy for numerical computation
import matplotlib.pyplot as plt #Matplotlib for data visualization
import seaborn as sns #Seaborn for data visualization

# Machine Learning
from sklearn.model_selection import train_test_split, KFold #To split the dataset and initialize kfold
from sklearn.metrics import mean_squared_error #For evaluation of mean_squared error
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder #For feature encoding of categorical data
import xgboost as xgb #Model
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Set random seed for reproducibility
np.random.seed(42)


LOADING DATASET

In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/hackathon-qualification/archive/train.csv") #Loading the train set
test_df = pd.read_csv("/content/drive/MyDrive/hackathon-qualification/archive/test.csv") #Loading the test set

In [ ]:
train_df.head(15) #To check the first 15 rows of the train set

In [ ]:
test_df.head(15) #To check the first 15 rows of the test set

In [ ]:
#Check the shape of the given datasets both train and test
print("Train DataFrame Shape:", train_df.shape)
print("Test DataFrame Shape:", test_df.shape)

In [ ]:
train_df.describe() #To check the mathematical features of the numerical columns in the train set

In [ ]:
test_df.describe() #To check the mathematical features of the numerical columns in the test set

In [ ]:
train_df.info() #To check the information of the columns in the train set

In [ ]:
test_df.info() #To check the information of the columns in the test set

EXPLORATORY DATA ANALYSIS

In [ ]:
print("DATA QUALITY ANALYSIS")
print("=" * 40)

#Missing Values Analysis
#Missing Values Analysis I Train set
print("MISSING VALUES IN TRAINING DATA:")
missing_train = train_df.isnull().sum()
print(missing_train[missing_train > 0])

#Missing Values Analysis in Test set
print("\n2. MISSING VALUES IN TEST DATA:")
missing_test = test_df.isnull().sum()
print(missing_test[missing_test > 0])

#Target Variable Analysis
print("\n3. TARGET VARIABLE 'price' ANALYSIS:")
print(f"   Min price: ${train_df['price'].min():,}")
print(f"   Max price: ${train_df['price'].max():,}")
print(f"   Mean price: ${train_df['price'].mean():,.0f}")
print(f"   Median price: ${train_df['price'].median():,.0f}")

#Unique Values in Categorical Columns
print("\n4. CATEGORICAL FEATURES UNIQUE COUNTS:")
categorical_cols = train_df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    print(f"   {col}: {train_df[col].nunique()} unique values")

DATA VISUALIZATION

In [ ]:
# Set style for plots
sns.set_style("whitegrid")

#Histograms for numerical features
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot histogram for model_year
sns.histplot(train_df['model_year'], bins=20, ax=axes[0])
axes[0].set_title('Distribution of Model Year')

# Plot histogram for mileage
sns.histplot(train_df['milage'], bins=20, ax=axes[1])
axes[1].set_title('Distribution of Milage')

# Plot histogram for price
sns.histplot(train_df['price'], bins=20, ax=axes[2])
axes[2].set_title('Distribution of Price')

plt.tight_layout()
plt.show()



In [ ]:
#Scatter plots vs price
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

#Scatter: model_year vs price
sns.scatterplot(x=train_df['model_year'], y=train_df['price'], alpha=0.5, ax=axes[0])
axes[0].set_title('Model Year vs Price')

#Scatter: mileage vs price
sns.scatterplot(x=train_df['milage'], y=train_df['price'], alpha=0.5, ax=axes[1])
axes[1].set_title('Milage vs Price')

plt.tight_layout()
plt.show()



In [ ]:
#Correlation heatmap for numerical features
numerical_df = train_df[['model_year', 'milage', 'price']]
corr_matrix = numerical_df.corr()

plt.figure(figsize=(6, 4))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Heatmap for Numerical Features')
plt.show()

In [ ]:
#Create boxplots for numerical features
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

#Boxplot for price
sns.boxplot(y=train_df['price'], ax=axes[0])
axes[0].set_title('Boxplot of Price')

# Boxplot for mileage
sns.boxplot(y=train_df['milage'], ax=axes[1])
axes[1].set_title('Boxplot of Mileage')

# Boxplot for model_year
sns.boxplot(y=train_df['model_year'], ax=axes[2])
axes[2].set_title('Boxplot of Model Year')

plt.tight_layout()
plt.show()

In [ ]:
#Function to detect outliers using IQR
def count_outliers(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = series[(series < lower_bound) | (series > upper_bound)]
    return len(outliers)

# Count outliers for price and mileage
price_outliers = count_outliers(train_df['price'])
milage_outliers = count_outliers(train_df['milage'])

print(f"Number of outliers in price: {price_outliers}")
print(f"Number of outliers in mileage: {milage_outliers}")

In [ ]:
from scipy.stats import skew

price_skew = skew(train_df['price']) #For evaluation of skewness
print(f"Skewness of price after cleaning: {price_skew}")

In [ ]:
# Calculate mean price for each brand
mean_price_by_brand = train_df.groupby('brand')['price'].mean().sort_values(ascending=False)

#Top 10 brands by mean price
top10_mean_price = mean_price_by_brand.head(10)

# Bar plot
plt.figure(figsize=(12, 6))
sns.barplot(x=top10_mean_price.index, y=top10_mean_price.values)
plt.xticks(rotation=45)
plt.title('Top 10 Brands by Average Price')
plt.ylabel('Average Price ($)')
plt.show()

In [ ]:
# Get count and mean price for top 10 brands by count
top10_brands_by_count = train_df['brand'].value_counts().head(10).index
train_top10 = train_df[train_df['brand'].isin(top10_brands_by_count)]

# Bar plot: mean price for top 10 brands (by count)
plt.figure(figsize=(12, 6))
sns.barplot(x='brand', y='price', data=train_top10, estimator='mean')
plt.xticks(rotation=45)
plt.title('Average Price for Top 10 Brands (by Frequency)')
plt.ylabel('Average Price ($)')
plt.show()

In [ ]:
# Fuel_type vs mean price
plt.figure(figsize=(8, 4))
sns.barplot(x='fuel_type', y='price', data=train_df, estimator='mean')
plt.xticks(rotation=45)
plt.title('Average Price by Fuel Type')
plt.show()

# Accident vs mean price
plt.figure(figsize=(6, 4))
sns.barplot(x='accident', y='price', data=train_df, estimator='mean')
plt.title('Average Price by Accident History')
plt.show()

# Transmission vs mean price (top 10 common transmissions)
top_transmissions = train_df['transmission'].value_counts().head(10).index
train_top_trans = train_df[train_df['transmission'].isin(top_transmissions)]
plt.figure(figsize=(10, 4))
sns.barplot(x='transmission', y='price', data=train_top_trans, estimator='mean')
plt.xticks(rotation=45)
plt.title('Average Price by Top 10 Transmissions')
plt.show()

DATA CLEANING

In [ ]:
print(" SMART DATA CLEANING")
print("=" * 40)

#Remove extreme price outliers (99.9th percentile)
price_upper_limit = train_df['price'].quantile(0.999)
train_clean = train_df[train_df['price'] <= price_upper_limit].copy()
outliers_removed = len(train_df) - len(train_clean)

print(f"1. Removed {outliers_removed} extreme price outliers")
print(f"   Price upper limit: ${price_upper_limit:,.0f}")

#Drop useless clean_title column (only 1 value)
train_clean = train_clean.drop('clean_title', axis=1)
test_df = test_df.drop('clean_title', axis=1)

print("2. Dropped 'clean_title' column (only 1 unique value)")

#Handle missing values
print("3. Missing values to handle later:")
print(f"   - fuel_type: {train_clean['fuel_type'].isnull().sum()} missing")
print(f"   - accident: {train_clean['accident'].isnull().sum()} missing")

print(f"\nNew training shape: {train_clean.shape}")
print(f" New test shape: {test_df.shape}")

 Strategic Missing Value Handling

In [ ]:
print(" SMART MISSING VALUE HANDLING")
print("=" * 40)

#Handle fuel_type missing values - use mode but be smart
most_common_fuel = train_clean['fuel_type'].mode()[0]
train_clean['fuel_type'] = train_clean['fuel_type'].fillna(most_common_fuel)
test_df['fuel_type'] = test_df['fuel_type'].fillna(most_common_fuel)

print(f"1. Filled fuel_type missing values with: '{most_common_fuel}'")

#Handle accident missing values - use 'None reported' (most common)
train_clean['accident'] = train_clean['accident'].fillna('None reported')
test_df['accident'] = test_df['accident'].fillna('None reported')

print("2. Filled accident missing values with: 'None reported'")

#Verify no more missing values
print("\n MISSING VALUES AFTER HANDLING:")
print("Training data:")
print(train_clean.isnull().sum())
print("\nTest data:")
print(test_df.isnull().sum())

print(f"\n FINAL CLEAN DATA SHAPES:")
print(f"Training: {train_clean.shape}")
print(f"Test: {test_df.shape}")

FEATURE ENCODING

In [ ]:
print(" ADVANCED FEATURE ENGINEERING")
print("=" * 40)

# Create new features for both training and test data
def create_features(df):
    df = df.copy()

    #Car Age (current year - model_year)
    df['car_age'] = 2025 - df['model_year']

    #Mileage per Year (usage intensity)
    df['miles_per_year'] = df['milage'] / df['car_age'].clip(lower=1)

    #Luxury Brand Flag (premium brands)
    luxury_brands = ['BMW', 'Mercedes-Benz', 'Audi', 'Lexus', 'Porsche', 'Genesis', 'Land Rover', 'Jaguar']
    df['is_luxury'] = df['brand'].isin(luxury_brands).astype(int)

    #Engine Power Extraction (if available in engine text)
    df['has_turbo'] = df['engine'].str.contains('Turbo|Twin|Supercharger', case=False, na=False).astype(int)

    return df

# Apply to both datasets
train_enhanced = create_features(train_clean)
test_enhanced = create_features(test_df)

print("CREATED POWERFUL NEW FEATURES:")
print("   - car_age (years since model year)")
print("   - miles_per_year (annual usage intensity)")
print("   - is_luxury (premium brand flag)")
print("   - has_turbo (high-performance engine flag)")

print(f"\n Enhanced Training Shape: {train_enhanced.shape}")
print(f" Enhanced Test Shape: {test_enhanced.shape}")

print("\n Sample of new features:")
display(train_enhanced[['brand', 'model_year', 'car_age', 'miles_per_year', 'is_luxury', 'has_turbo']].head(3))

 Prepare Data for Modeling

In [ ]:
print(" PREPARING DATA FOR MODELING")
print("=" * 40)

#Separate features (X) and target (y) for training
X = train_enhanced.drop('price', axis=1)
y = train_enhanced['price']

print(" FEATURES (X) and TARGET (y) separated:")
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

#Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f"\n Categorical features ({len(categorical_cols)}):")
print(categorical_cols)

print(f"\n Numerical features ({len(numerical_cols)}):")
print(numerical_cols)

print(f"\n New engineered features:")
new_features = ['car_age', 'miles_per_year', 'is_luxury', 'has_turbo']
for feat in new_features:
    print(f"   - {feat}")

Remove ID Column & Retrain on FULL DATA

In [ ]:

print(" REMOVING PROBLEMATIC ID COLUMN & TRAINING ON FULL DATA")
print("=" * 50)

# Remove ID column from all datasets
X_encoded_no_id = X_encoded.drop('id', axis=1)
test_encoded_no_id = test_encoded.drop('id', axis=1)

print(" ID COLUMN REMOVED!")
print(f"New X shape: {X_encoded_no_id.shape}")
print(f"New test shape: {test_encoded_no_id.shape}")

# Train on FULL dataset (no splitting)
print("\n TRAINING XGBOOST ON FULL DATASET (NO SPLITTING)...")

xgb_model_full = xgb.XGBRegressor(
    n_estimators=2500,
    learning_rate=0.01,
    max_depth=5,
    subsample=0.6,
    colsample_bytree=0.8,
    random_state=0,
    n_jobs=-1,
    eval_metric='rmse'
)

# Train on ALL data (no validation split)
xgb_model_full.fit(X_encoded_no_id, y, verbose=50)

print("XGBOOST TRAINING COMPLETED ON FULL DATASET!")

#CREATE SUBMISSION
print(" CREATING SUBMISSION FILE")
print("=" * 40)

# Make predictions on the test set
print("Making predictions on test data...")
test_predictions = xgb_model_full.predict(test_encoded_no_id)

print(f"Predictions range: ${test_predictions.min():,.2f} - ${test_predictions.max():,.2f}")
print(f"Mean prediction: ${test_predictions.mean():,.2f}")

# Create submission DataFrame
submission = pd.DataFrame({
    'id': test_df['id'],  # Use the original ID column from test data
    'price': test_predictions
})

# Save the submission file
submission_file = 'submission_xgboost_full.csv'
submission.to_csv(submission_file, index=False)
print(f"Submission saved as '{submission_file}'")

# Verify the submission
print(f"Submission shape: {submission.shape}")
print(f" Number of predictions: {len(submission)}")

print(f"\n SUBMISSION READY: {submission_file}")

In [ ]:
# ESTIMATE RMSE USING TRAINING DATA
print(" ESTIMATING RMSE")
print("=" * 40)

# Make predictions on the training data
train_predictions = xgb_model_full.predict(X_encoded_no_id)

# Calculate RMSE on training data
train_rmse = np.sqrt(mean_squared_error(y, train_predictions))

print(f"Training RMSE: ${train_rmse:,.2f}")

# This gives you an estimate of how well your model performs
# Note: This will usually be optimistic (lower than actual test RMSE)
# A good rule of thumb is to add 5-15% to this estimate for test performance

estimated_test_rmse = train_rmse * 1.1  # Add 10% buffer
print(f"Estimated Test RMSE: ${estimated_test_rmse:,.2f} (with 10% buffer)")

# ADDITIONAL PERFORMANCE METRICS
from sklearn.metrics import mean_absolute_error, r2_score

train_mae = mean_absolute_error(y, train_predictions)
train_r2 = r2_score(y, train_predictions)

print(f"Training MAE: ${train_mae:,.2f}")
print(f"Training R²: {train_r2:.4f}")

#ANALYZE PREDICTION ACCURACY
print("\n PREDICTION ACCURACY ANALYSIS:")
print("=" * 40)

# Calculate error statistics
errors = y - train_predictions
abs_errors = np.abs(errors)

print(f"Mean Error: ${errors.mean():,.2f}")
print(f"Mean Absolute Error: ${abs_errors.mean():,.2f}")
print(f"Max Overprediction: ${errors.min():,.2f}")
print(f"Max Underprediction: ${errors.max():,.2f}")

